In [ ]:
from arthurai import ArthurAI
from arthurai.client.apiv3 import InputType, OutputType, Stage
import numpy as np
import joblib
import datetime
import time

In [ ]:
import sys
sys.path.append("..")
from model_utils import transformations, load_datasets

In this guide, we'll use the credit dataset (and a pre-trained model) to onboard a new model to the Arthur platform. We'll walk through registering the model using a sample of the training data. This is an example of a streaming model.

#### Set up connection
Supply your API Key below to autheticate with the platform.

In [ ]:
URL = "app.arthur.ai"
ACCESS_KEY = "..."

connection = ArthurAI(url=URL, access_key=ACCESS_KEY, client_version=3)

## Create Model

We'll instantiate a model object with a small amount of metadata about the model input and output types. Then, we'll use a sample of the training data to register the full data schema for this Tabular model.

In [ ]:
arthur_model = connection.model(partner_model_id="CreditRiskModel_v0.0.1",
                               input_type=InputType.Tabular,
                               output_type=OutputType.Multiclass)

In [ ]:
(X_train, Y_train), (X_test, Y_test) = load_datasets("../fixtures/datasets/credit_card_default.csv")

In [ ]:
Y_train.head()

In [ ]:
X_train.head()

We need to register what the data schema is for the inputs to the model. Since your model might hundreds or thousands of input features, you can just pass us a pandas DataFrame of your training data, and we'll handle the rest.

In [ ]:
arthur_model.from_dataframe(X_train, Stage.ModelPipelineInput)

We need to register the schema for the outputs of the model: what will a typical prediction look like and what will a typical ground truth look like? What names, shapes, and datatypes should Arthur expect for these objects?

Since this is a binary classification model, we'll do this all in one step with the *.add_binary_classifier_output_attributes()* method. All we need to supply is a mapping that establishes:
  * names for the model's predictions
  * names for the model's ground truth
  * the mapping that related these two
  
Our classifier will be making predictions about class *0* and class *1* and will return a probability score for each class. Therefore, we'll set up a name *prediction_0* and a name *prediction_1*. Additionally, our groundtruth will be either a 0 or 1, but we'll always represent ground truth in the one-hot-endoded form. Therefore, we create two field called *gt_0* and *gt_1*. We link these all up in a dictionary and pass that to the model.  

In [ ]:
prediction_to_ground_truth_map = {
    "prediction_0": "gt_0",
    "prediction_1": "gt_1"
}

arthur_model.add_binary_classifier_output_attributes("prediction_1", prediction_to_ground_truth_map)

Note that the first argument to *.add_binary_classifier_output_attributes()* is the name of the "positive predicted class", for purposes of calculating accuracy metrics. 

Before saving, you can review a model to make sure everything is correct.

In [ ]:
arthur_model.review()

In [ ]:
arthur_model.save()

### Setting baseline data
Next, we'll use the training data to set a baseline reference for calcuating data drift. 

For tracking data drift, you can upload a dataset to serve as the baseline or reference set. Often, this is a sample of your training data for the associated model. Our reference dataset should ideally include examples of
  * inputs 
  * ground truth
  * model predictions
  
for a sample of the training set. This way, Arthur can monitor for drift and stability in all of these aspects. 

In [ ]:
# load our pre-trained classifier so we can generate predictions
sk_model = joblib.load("../fixtures/serialized_models/credit_model.pkl")

In [ ]:
# get all input columns
reference_set = X_train.copy()

# get ground truth labels
reference_set["gt_1"] = Y_train
reference_set["gt_0"] = 1-Y_train

# get model predictions
preds = sk_model.predict_proba(X_train)
reference_set["prediction_1"] = preds[:, 1]
reference_set["prediction_0"] = preds[:, 0]


In [ ]:
arthur_model.set_reference_data(data=reference_set)

## Sending Inferences

Load test data and trained model. Let's familiarize ourselves with the data and the model.


In [ ]:
X_test.shape

In [ ]:
sk_model

In [ ]:
sk_model.predict_proba(X_train.iloc[0:1, :])

To send inferences, we'll iterate through datapoints in a test set and send telemetry to Arthur. You can send inferences one at a time or in a list. We will combine our model inputs and our model predictions into a dictionary called *inference_data*. 

In [ ]:
for i in range(X_test.shape[0]):
    datarecord = X_test.iloc[i:i+1, :]
    predicted_probs = sk_model.predict_proba(datarecord)[0]
    ground_truth = np.int(Y_test.iloc[i])
    external_id = str(np.random.randint(1e9))

    inputs = datarecord.to_dict(orient='records')[0]
    prediction = {"prediction_1":predicted_probs[1], 
                  "prediction_0":predicted_probs[0]}
    ground_truth={"gt_1": ground_truth, 
                  "gt_0":1-ground_truth}

    
    arthur_model.send_inferences([{
        "inference_timestamp" : datetime.datetime.utcnow().isoformat() + "Z",
        "partner_inference_id" : external_id,
        "inference_data": inputs.update(prediction),
        "ground_truth_data": ground_truth,
        "ground_truth_timestamp" : datetime.datetime.utcnow().isoformat() + "Z"
    }])
    
    print("Sent inference with id {}".format(external_id))
    time.sleep(0.001 * np.random.random())

You can send inferences one at a time but you can also send them in small bunches using the *send_infereces()* method. In that case, you would send a list of dictionaries, each of which is similar to above. 

If you model scoring system is a set up in a batch processor where you run a daily, weekly, or monthly job, then we recommend setting a batch model with Arthur and using the corresponding *send_batch_inferences()* method. An example batch model can be found [here](../../credit_risk_batch/notebooks/Quickstart.ipynb).